In [1]:
import os
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torchvision import transforms
import pickle

In [2]:
def Patch_Generator(img_loc):
    im = Image.open(img_loc).convert('RGB') #PIL object in RGB
    h,w = im.size
    
    all_patches = torch.empty((1,3,40,40))
    
    for scale in [1, 0.9, 0.8, 0.7]:
        img = transforms.Compose([transforms.Resize((int(scale*h),int(scale*w))),transforms.ToTensor()])(im) #of shape (1,scale*h,scale*w)

        kh, kw = 40, 40  # kernel size
        dh, dw = 10, 10  # stride

        patches = img.unfold(1, kh, dh).unfold(2, kw, dw) #of shape (3,h_patch_patch,w_patch_num,kh,kw)
        patches = patches.permute((2,1,0,3,4)).reshape(-1,3,kh, kw) #of shape (num_patches,3,kh,kw)

        patches2 = torch.flipud(patches.permute((2,3,1,0))) #of shape (kh,kw,3,num_patches)
        patches2 = patches2.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches3 = torch.rot90(patches.permute((2,3,1,0)),k=1) #of shape (kh,kw,3,num_patches)
        patches3 = patches3.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches4 = torch.flipud(torch.rot90(patches.permute((2,3,1,0)),k=1)) #of shape (kh,kw,3,num_patches)
        patches4 = patches4.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches5 = torch.rot90(patches.permute((2,3,1,0)),k=2) #of shape (kh,kw,3,num_patches)
        patches5 = patches5.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches6 = torch.flipud(torch.rot90(patches.permute((2,3,1,0)),k=2)) #of shape (kh,kw,3,num_patches)
        patches6 = patches6.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches7 = torch.rot90(patches.permute((2,3,1,0)),k=3) #of shape (kh,kw,3,num_patches)
        patches7 = patches7.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        patches8 = torch.flipud(torch.rot90(patches.permute((2,3,1,0)),k=3)) #of shape (kh,kw,3,num_patches)
        patches8 = patches8.permute((3,2,0,1)) #of shape (num_patches,3,kh,kw)

        all_patches = torch.concat([all_patches,patches,patches2,patches3,patches4,patches5,patches6,patches7,patches8],dim=0)
    
    return all_patches[1:] #of shape (total_num_patches,3,kh,kw)


In [3]:
img_augs = Patch_Generator('CroppedImages/Sony_4-5_125_3200_plant_13_real.JPG')
    

In [4]:
print("The number of augmentations generated is "+str(img_augs.shape[0]-1))

The number of augmentations generated is 52367


In [5]:
dbfile = open('PolyU_train.pkl', 'wb')
# source, destination
pickle.dump(img_augs[1:], dbfile)                    
dbfile.close()
